In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

## Building/Cleaning Data to Create CSV/DataFrames

In [2]:
df = pd.read_csv('data/Indonesia/Indonesia.csv')

In [3]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
# missing value for 2020
# filling with 75th percentile

df['Ind_prod_60kg_bags'].fillna(df['Ind_prod_60kg_bags'].quantile(.75), inplace=True)

In [ ]:
df['ind_prod_lbs'] = [(each*1000)*2.2 for each in df['Ind_prod_60kg_bags']]

In [9]:
df.drop('Ind_prod_60kg_bags', axis=1, inplace=True)

In [12]:
# df.to_csv('data/Indonesia/Indonesia2.csv')

In [197]:
avg_temp = pd.read_csv('data/Indonesia/tas_timeseries_annual_cru_1901-2021_IDN.csv')

In [199]:
avg_temp.drop(avg_temp.columns[[1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34]], axis=1, inplace=True)

In [200]:
avg_temp.drop(avg_temp.columns[[2]], axis=1, inplace=True)

In [202]:
avg_temp2 = avg_temp.drop(avg_temp.index[0:89])

In [204]:
avg_temp3 = avg_temp2.drop(avg_temp2.index[31])

In [208]:
avg_temp3.rename(columns={'Variable:': 'Year'}, inplace=True)

In [217]:
avg_temp3.rename(columns={'Indonesia': 'ind_avg_temp_celsius'}, inplace=True)

In [220]:
ind = pd.merge(df, avg_temp3, on='Year')

In [222]:
ind = ind[['Year', 'Temp Change', 'ind_avg_temp_celsius', 'ind_prod_lbs']]

In [224]:
ind.rename(columns={'Year': 'year', 'Temp Change': 'temp_change'}, inplace=True)

In [227]:
ind.to_csv('data/Indonesia/ind_final.csv')

In [2]:
df = pd.read_csv('data/Indonesia/ind_final.csv')

In [4]:
df.drop(df.columns[[0]], axis=1, inplace=True)

In [6]:
df2 = pd.read_csv('data/cleaned_merged_df.csv')

In [11]:
df.rename(columns={'temp_change': 'ind_annual_celcsius_change'}, inplace=True)

In [16]:
cleaned_merged_df2 = pd.merge(df, df2, on='year')

In [18]:
# cleaned_merged_df2

In [19]:
cleaned_merged_df2.to_csv('data/cleaned_merged_df2.csv')

In [54]:
b_prod = pd.read_csv('data/production.csv')

In [55]:
b_prod.drop(b_prod.columns[[0]], axis=1, inplace=True)

In [57]:
b_prod.drop(b_prod.index[0:2], inplace=True)

In [59]:
b_prod.drop(b_prod.index[1:56], inplace=True)

In [66]:
df = b_prod.transpose()

In [68]:
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

In [78]:
df = df.reset_index()

In [80]:
df.rename(columns={'index': 'year', 'Brazil': 'b_prod_60kg_bags'}, inplace=True)

In [84]:
df["year"]= df["year"].str.split("/", n = 1, expand = False)

In [83]:
df

Crop year,year,b_prod_60kg_bags
0,"[1990, 91]",27285.6286
1,"[1991, 92]",27293.4934
2,"[1992, 93]",34603.3542
3,"[1993, 94]",28166.9786
4,"[1994, 95]",28192.047
5,"[1995, 96]",18060.2022
6,"[1996, 97]",29196.743
7,"[1997, 98]",26148.004
8,"[1998, 99]",36760.8533
9,"[1999, 00]",47577.8065


# Regression Model

### Loading Data and Train/Test Split

In [37]:
df = pd.read_csv('data/cleaned_merged_df3.csv')

In [38]:
df.drop(df.columns[[0, 1]], axis=1, inplace=True)

In [39]:
df.head(3)

,year,ind_annual_celcsius_change,ind_avg_temp_celsius,ind_prod_lbs,b_hectares_harvested,b_hectograms_per_hectare_yield,b_tonnes_produced,usa_retail_price,b_annual_avg_temp,col_production_lbs,col_annual_celsius_change,global_change_in_celsius,e_prod,e_temp_change,e_export,e_consumption,us_consumption_lbs,col_avg_temp_celsius,b_annual_celsius_change,e_avg_temp
0,1990,0.120,25.43,16371042.6,2908960,5036,1464856,2.97,25.12,31671200.0,0.258,0.66,2909.451,0.244,1074.101,1200.0,40254757.4,24.77,0.440,23.04
1,1991,0.203,25.32,18685031.2,2763440,5502,1520382,2.81,25.05,39556873.4,0.347,0.53,2924.664,0.244,841.329,1264.0,41054596.0,24.82,0.470,23.13
2,1992,0.264,25.28,12252851.6,2500320,5177,1294373,2.58,24.91,32883131.6,0.532,0.25,1825.171,0.244,734.461,1331.0,40238996.6,24.95,0.381,22.79


In [25]:
temp_features = [
    'ind_avg_temp_celsius', 
    'e_avg_temp', 
    'col_avg_temp_celsius',
    'b_annual_avg_temp'
]

In [ ]:
prod_features = [
    'ind_prod_lbs', 
    'col_production_lbs', 
    'e_prod',
    'b_annual_avg_temp'
]

In [26]:
df[temp_features].isnull().sum()

ind_avg_temp_celsius    0
e_avg_temp              0
col_avg_temp_celsius    0
b_annual_avg_temp       0
dtype: int64

In [27]:
X = df[temp_features]
y = df['usa_retail_price']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### Scaling

In [28]:
ss = StandardScaler()
print(ss.fit(X_train))

StandardScaler()


In [29]:
ss = StandardScaler()
print(ss.fit(X_test))

StandardScaler()


In [30]:
mlr = LinearRegression()
mlr.fit(X_train, y_train)

y_pred = mlr.predict(X_test)

print('MLR score:', mlr.score(X_train, y_train), mlr.score(X_test, y_test))
print('MLR intercept:', mlr.intercept_)
print('MLR coeffs:', mlr.coef_)

MLR score: 0.37277105117546516 -0.46385685051501957
MLR intercept: -31.36281896027752
MLR coeffs: [-1.26214349 -0.40741388 -0.18179427  3.20308214]
